# Agents as Tools

**Building a Hierarchical Customer Service System with Specialized Agents**

---

Welcome to this comprehensive tutorial on **Agents as Tools** in the Strands framework! This notebook demonstrates how to transform specialized AI agents into callable tools that can be used by an orchestrator agent. By the end of this 10-minute tutorial, you'll build a complete customer service system that routes emails to the right specialists.

### 🎯 What You'll Learn

In this tutorial, you will:
- Transform specialized agents into callable tools
- Build an orchestrator agent that delegates to specialists
- Create a hierarchical customer service system
- Handle complex multi-topic customer inquiries
- Apply best practices for agent tool design

### 📧 Our Use Case: Customer Service Email Handler

Imagine a customer service department where:
- A **manager** (orchestrator agent) reads incoming emails
- The manager identifies the type of inquiry
- The email is routed to the appropriate **specialist**
- The specialist handles the specific request
- The customer receives expert assistance

### 🏗️ Why Agents as Tools?

This pattern offers powerful benefits:
- **Separation of Concerns**: Each agent focuses on its expertise
- **Scalable Architecture**: Easy to add new specialists
- **Improved Performance**: Tailored prompts per domain
- **Maintainable Systems**: Modular design
- **Real-world Modeling**: Mirrors actual team structures

## 📦 Step 1: Installing Required Packages

### Overview
Let's install the necessary packages for building our customer service system.

### 📚 Packages We'll Install
- **strands-agents**: Core framework with agent and tool support
- **strands-agents-tools**: Additional tools for our agents
- **boto3**: For AWS Bedrock integration

In [ ]:
# Install required packages
%pip install strands-agents strands-agents-tools boto3 -q

print("✅ All packages installed successfully!")
print("   Ready to build our customer service team! 👥")

## 🔐 Step 2: Setting Up AWS Authentication

### Overview
We'll configure AWS Bedrock for our agent team.

### 🔑 Authentication Options
1. **AWS Profile** (Recommended for development)
2. **Environment Variables**
3. **Direct Credentials** (Less secure)
4. **IAM Roles** (Recommended for production)

In [ ]:
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from typing import List, Dict, Any
import json

# Configure AWS session
session = boto3.Session(
    # aws_access_key_id='your_access_key',
    # aws_secret_access_key='your_secret_key',
    # aws_session_token='your_session_token',  # If using temporary credentials
    # region_name='us-west-2',
    profile_name='default'  # Optional: Use a specific AWS profile
)

# Create a Bedrock model instance
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    boto_session=session
)

print("✅ AWS Bedrock configured successfully!")
print(f"   Model: Claude 3.7 Sonnet")
print(f"   Profile: {session.profile_name}")

## 👤 Step 3: Creating Our First Specialist Agent Tool

### The Order Tracking Specialist

Let's start by creating our first specialist - an order tracking expert. This agent will handle all inquiries related to:
- Order status checks
- Shipping updates
- Delivery issues
- Package tracking

In [ ]:
# Define the Order Tracking Specialist as a tool
@tool
def order_tracking_agent(query: str) -> str:
    """
    Handles order status inquiries, shipping updates, and delivery issues.
    This specialist has access to order databases and shipping information.
    
    Args:
        query: A customer inquiry about their order
        
    Returns:
        A detailed response about the order status
    """
    try:
        # Create a specialized agent for order tracking
        agent = Agent(
            model=bedrock_model,
            system_prompt="""You are an order tracking specialist for our e-commerce company.
            You have access to order databases and shipping carrier systems.
            
            Your responsibilities:
            - Provide order status updates
            - Track packages and shipping
            - Explain delivery timelines
            - Handle lost or delayed packages
            
            Always be helpful and provide specific information.
            If an order number is mentioned, acknowledge it.
            Format your responses clearly with order details."""
        )
        
        # Process the customer query
        response = agent(query)
        return str(response)
        
    except Exception as e:
        return f"Error in order tracking system: {str(e)}"

# Test our first specialist
test_response = order_tracking_agent("Where is my order #12345?")
print("📦 Order Tracking Specialist Response:")
print(test_response)

## 👥 Step 4: Building the Complete Customer Service Team

### Our Specialized Team

Now let's create our full team of customer service specialists:

1. **Order Tracking Agent** - Already created above
2. **Technical Support Agent** - Product issues and troubleshooting
3. **Billing Agent** - Payments, refunds, and invoices
4. **General Inquiry Agent** - FAQs and company information

In [ ]:
# Technical Support Specialist
@tool
def technical_support_agent(query: str) -> str:
    """
    Handles technical issues, product troubleshooting, and setup assistance.
    This specialist understands product specifications and common problems.
    
    Args:
        query: A technical issue or product question
        
    Returns:
        Technical support and troubleshooting guidance
    """
    try:
        agent = Agent(
            model=bedrock_model,
            system_prompt="""You are a technical support specialist.
            You have deep knowledge of our products and common technical issues.
            
            Your responsibilities:
            - Troubleshoot product problems
            - Guide customers through setup processes
            - Explain product features and specifications
            - Provide step-by-step solutions
            
            Always provide clear, patient instructions.
            Use numbered steps when explaining procedures."""
        )
        
        response = agent(query)
        return str(response)
        
    except Exception as e:
        return f"Error in technical support: {str(e)}"

# Billing Specialist
@tool
def billing_agent(query: str) -> str:
    """
    Handles payment issues, refunds, invoices, and billing inquiries.
    This specialist manages financial transactions and account matters.
    
    Args:
        query: A billing or payment-related question
        
    Returns:
        Billing information and payment assistance
    """
    try:
        agent = Agent(
            model=bedrock_model,
            system_prompt="""You are a billing and payments specialist.
            You handle all financial aspects of customer accounts.
            
            Your responsibilities:
            - Process refund requests
            - Explain charges and invoices
            - Handle payment failures
            - Update payment methods
            
            Be professional when discussing financial matters.
            Always confirm understanding of monetary amounts."""
        )
        
        response = agent(query)
        return str(response)
        
    except Exception as e:
        return f"Error in billing system: {str(e)}"

# General Inquiry Specialist
@tool
def general_inquiry_agent(query: str) -> str:
    """
    Handles general questions about company policies, hours, and FAQs.
    This specialist provides general information and guidance.
    
    Args:
        query: A general question about the company
        
    Returns:
        General information and guidance
    """
    try:
        agent = Agent(
            model=bedrock_model,
            system_prompt="""You are a general customer service representative.
            You handle FAQs and provide general company information.
            
            Your responsibilities:
            - Answer questions about business hours
            - Explain company policies
            - Provide contact information
            - Handle general inquiries
            
            Be friendly and informative.
            Provide comprehensive answers to common questions."""
        )
        
        response = agent(query)
        return str(response)
        
    except Exception as e:
        return f"Error in general inquiry system: {str(e)}"

print("✅ Customer Service Team Created!")
print("   📦 Order Tracking Specialist")
print("   🔧 Technical Support Specialist")
print("   💳 Billing Specialist")
print("   ℹ️  General Inquiry Specialist")

## 🎯 Step 5: Creating the Customer Service Manager (Orchestrator)

### The Orchestrator Agent

Now let's create the manager who reads customer emails and routes them to the appropriate specialist. This orchestrator agent has access to all specialists as tools.

In [ ]:
# Create the Customer Service Manager (Orchestrator)
customer_service_manager = Agent(
    model=bedrock_model,
    system_prompt="""You are a customer service manager responsible for routing customer emails to the appropriate specialist.
    
    Your team consists of:
    - order_tracking_agent: For order status, shipping, delivery questions
    - technical_support_agent: For product issues, troubleshooting, setup help
    - billing_agent: For payments, refunds, invoices, charges
    - general_inquiry_agent: For company info, policies, hours, general questions
    
    Your job is to:
    1. Read and understand the customer's email
    2. Identify the type of inquiry
    3. Route to the appropriate specialist using their tool
    4. Return the specialist's response
    
    If an email covers multiple topics, you may need to consult multiple specialists.
    Always choose the most specific specialist for each part of the inquiry.""",
    tools=[
        order_tracking_agent,
        technical_support_agent,
        billing_agent,
        general_inquiry_agent
    ]
)

print("✅ Customer Service Manager Created!")
print("   The orchestrator can now route emails to:")
print("   - Order Tracking")
print("   - Technical Support")
print("   - Billing")
print("   - General Inquiries")

## 📧 Step 6: Processing Real Customer Emails

### Testing Our System

Let's test our customer service system with various types of customer emails. Watch how the manager correctly routes each inquiry to the right specialist.

In [ ]:
# Test Email 1: Order Tracking
print("📧 EMAIL 1: Order Tracking")
print("-" * 50)
email1 = "Where is my order #12345? I ordered it 3 days ago and haven't received any updates."
print(f"Customer: {email1}")
print("\n🤖 Manager Processing...")

response1 = customer_service_manager(email1)
print(f"\n📬 Response: {response1}")
print("\n" + "=" * 70 + "\n")

In [ ]:
# Test Email 2: Technical Support
print("📧 EMAIL 2: Technical Support")
print("-" * 50)
email2 = "My product stopped working after 2 days. The screen won't turn on. What should I do?"
print(f"Customer: {email2}")
print("\n🤖 Manager Processing...")

response2 = customer_service_manager(email2)
print(f"\n📬 Response: {response2}")
print("\n" + "=" * 70 + "\n")

In [ ]:
# Test Email 3: Billing
print("📧 EMAIL 3: Billing")
print("-" * 50)
email3 = "I was charged twice for my order. Can I get a refund for the duplicate charge?"
print(f"Customer: {email3}")
print("\n🤖 Manager Processing...")

response3 = customer_service_manager(email3)
print(f"\n📬 Response: {response3}")
print("\n" + "=" * 70 + "\n")

In [ ]:
# Test Email 4: General Inquiry
print("📧 EMAIL 4: General Inquiry")
print("-" * 50)
email4 = "What are your business hours? Do you have a physical store I can visit?"
print(f"Customer: {email4}")
print("\n🤖 Manager Processing...")

response4 = customer_service_manager(email4)
print(f"\n📬 Response: {response4}")
print("\n" + "=" * 70 + "\n")

## 🔄 Step 7: Handling Complex Multi-Topic Emails

### When Customers Have Multiple Issues

Real customer emails often contain multiple questions or issues. Let's see how our orchestrator handles complex inquiries that require multiple specialists.

In [ ]:
# Complex Email: Multiple Issues
print("📧 COMPLEX EMAIL: Multiple Topics")
print("-" * 50)
complex_email = """My order #12345 arrived yesterday but the product isn't working properly. 
The screen flickers and then shuts off. Can I get a replacement or a refund? 
Also, what's your warranty policy for defective items?"""

print(f"Customer: {complex_email}")
print("\n🤖 Manager Processing Complex Inquiry...")
print("   (This may require multiple specialists)")

complex_response = customer_service_manager(complex_email)
print(f"\n📬 Response: {complex_response}")

## 💡 Step 8: Best Practices & Tips

### Building Effective Agent Tools

Let's explore the best practices for creating agent tools and orchestrator systems.

In [ ]:
# Best Practices Demonstration
print("💡 BEST PRACTICES FOR AGENTS AS TOOLS")
print("=" * 60)

best_practices = {
    "📝 Clear Tool Documentation": [
        "Write descriptive docstrings explaining the agent's expertise",
        "Specify what types of queries the agent handles",
        "Document expected input and output formats"
    ],
    "🎯 Focused System Prompts": [
        "Keep each specialist tightly focused on its domain",
        "Define clear responsibilities and boundaries",
        "Include specific examples in the prompt"
    ],
    "📊 Consistent Response Formats": [
        "Ensure all specialists return similar response structures",
        "Use consistent tone and formatting",
        "Handle errors gracefully with informative messages"
    ],
    "🔀 Clear Routing Logic": [
        "Give the orchestrator explicit routing criteria",
        "List each specialist and when to use them",
        "Handle edge cases and multiple topics"
    ],
    "🧪 Testing Strategy": [
        "Test each specialist individually first",
        "Test the orchestrator with various query types",
        "Include edge cases and complex scenarios"
    ]
}

for practice, tips in best_practices.items():
    print(f"\n{practice}")
    for tip in tips:
        print(f"   • {tip}")

In [ ]:
# Example: Creating a Well-Documented Agent Tool
@tool
def returns_and_exchanges_agent(query: str) -> str:
    """
    Specialist for handling returns, exchanges, and warranty claims.
    
    Expertise:
    - Return policy and procedures
    - Exchange processes
    - Warranty claims
    - RMA (Return Merchandise Authorization) requests
    
    Input: Customer query about returning or exchanging products
    Output: Detailed instructions and policy information
    
    Examples of handled queries:
    - "How do I return a defective item?"
    - "Can I exchange this for a different size?"
    - "What's covered under warranty?"
    """
    try:
        agent = Agent(
            model=bedrock_model,
            system_prompt="""You are a returns and exchanges specialist.
            
            Key policies to remember:
            - 30-day return window for unopened items
            - 14-day exchange period for wrong sizes
            - 1-year warranty on all electronics
            - Free return shipping for defective items
            
            Always provide step-by-step return instructions.
            Be empathetic to customer concerns."""
        )
        
        response = agent(query)
        return str(response)
        
    except Exception as e:
        return f"Returns system temporarily unavailable. Please try again."

print("✅ Example of a well-documented agent tool created!")
print("   Notice the detailed docstring and error handling")

## 🚀 Putting It All Together

### Complete Customer Service System

Let's create a function that processes a batch of customer emails to demonstrate our complete system in action.

In [ ]:
def process_customer_emails(emails: List[Dict[str, str]]) -> List[Dict[str, Any]]:
    """
    Process a batch of customer emails through our service system.
    
    Args:
        emails: List of email dictionaries with 'id' and 'content'
        
    Returns:
        List of processed emails with responses
    """
    results = []
    
    for email in emails:
        print(f"\n📧 Processing Email ID: {email['id']}")
        print(f"   Subject: {email.get('subject', 'No subject')}")
        
        try:
            # Process through the orchestrator
            response = customer_service_manager(email['content'])
            
            results.append({
                'email_id': email['id'],
                'subject': email.get('subject', 'No subject'),
                'status': 'processed',
                'response': response
            })
            
            print("   ✅ Successfully processed")
            
        except Exception as e:
            results.append({
                'email_id': email['id'],
                'subject': email.get('subject', 'No subject'),
                'status': 'error',
                'error': str(e)
            })
            
            print(f"   ❌ Error: {str(e)}")
    
    return results

# Simulate a batch of customer emails
customer_email_batch = [
    {
        'id': 'E001',
        'subject': 'Order Status',
        'content': 'I placed an order yesterday but haven\'t received a confirmation. Order #98765.'
    },
    {
        'id': 'E002',
        'subject': 'Product Not Working',
        'content': 'The wireless headphones I bought last week won\'t connect to my phone anymore.'
    },
    {
        'id': 'E003',
        'subject': 'Question about your store',
        'content': 'Do you offer gift wrapping services? I want to send a birthday present.'
    }
]

# Process the batch
print("🚀 PROCESSING CUSTOMER EMAIL BATCH")
print("=" * 60)

processed_emails = process_customer_emails(customer_email_batch)

# Summary
print("\n📊 BATCH PROCESSING SUMMARY")
print("=" * 60)
print(f"Total emails processed: {len(processed_emails)}")
print(f"Successful: {sum(1 for e in processed_emails if e['status'] == 'processed')}")
print(f"Errors: {sum(1 for e in processed_emails if e['status'] == 'error')}")

## 🎉 Congratulations!

### 🏆 What You've Accomplished

In this tutorial, you've successfully:
- ✅ Transformed specialized agents into callable tools
- ✅ Built an orchestrator agent that delegates work
- ✅ Created a complete customer service system
- ✅ Handled complex multi-topic inquiries
- ✅ Applied best practices for agent tool design

### 🏢 The Power of Hierarchical Agent Systems

You now have the skills to:
- **Build teams of specialized AI agents** that work together
- **Create orchestrator agents** that route tasks intelligently
- **Scale your systems** by adding new specialists
- **Handle complex queries** through delegation
- **Model real-world organizations** with AI

### 💡 Key Takeaways

1. **Separation of Concerns**: Each agent focuses on its expertise
2. **Clear Documentation**: Good docstrings help orchestrators route correctly
3. **Focused Prompts**: Specialized agents perform better than generalists
4. **Error Handling**: Always handle failures gracefully
5. **Testing Strategy**: Test individual agents before integration

### 🚀 Where to Go From Here

Consider exploring:
- **Adding More Specialists**: Shipping agents, loyalty program agents, etc.
- **Nested Hierarchies**: Department heads managing team leads
- **State Management**: Tracking customer history across interactions
- **Performance Optimization**: Caching and parallel processing
- **Production Deployment**: Scaling your customer service system

### 📚 Resources

- [Strands Documentation](https://strandsagents.com/0.1.x/)
- [Multi-Agent Patterns](https://strandsagents.com/0.1.x/user-guide/concepts/multi-agent/)
- [Tool Development Guide](https://strandsagents.com/0.1.x/user-guide/concepts/tools/)

### 🌟 Next Steps

You're ready to:
1. Design hierarchical agent systems for your use cases
2. Build specialized agents for different domains
3. Create intelligent orchestration logic
4. Deploy multi-agent systems in production
5. Lead AI architecture initiatives

### 🎊 Final Thoughts

The "Agents as Tools" pattern transforms how we build AI systems. By creating teams of specialized agents coordinated by intelligent orchestrators, you can tackle complex problems that would overwhelm a single agent.

Remember: Great AI systems are built one specialized agent at a time!

Happy building! 🤖🛠️✨